In [1]:
!pip install stable-baselines3[extra]

In [2]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...


In [3]:
import requests
import sys
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import torch

In [4]:
sys.path.insert(0,'boptestGymService')

url = 'https://api.boptest.net'

In [5]:
from boptestGymService.boptestGymEnv import BoptestGymEnv
from boptestGymEnv import DiscretizedActionWrapper

C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:122: SyntaxWarning: invalid escape sequence '\c'
  axs[0].set_ylabel('Operative\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:131: SyntaxWarning: invalid escape sequence '\c'
  axs[3].set_ylabel('Ambient\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:135: SyntaxWarning: invalid escape sequence '\d'
  axt.plot(x_time, df['weaSta_reaWeaHDirNor_y'], color='gold', linestyle='-', linewidth=1, label='$\dot{Q}_rad$')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:141: SyntaxWarning: invalid escape sequence '\d'
  axs[3].plot([],[], color='gold',        linestyle='-', linewidth=1, label='$\dot{Q}_{rad}$')


In [6]:
env = BoptestGymEnv(url                   = url,
                    testcase              = 'bestest_hydronic_heat_pump',
                    actions               = ['oveHeaPumY_u'],
                    observations          = {'reaTZon_y':(280.,310.)},
                    random_start_time     = False,
                    start_time            = 31*24*3600,
                    max_episode_length    = 24*3600,
                    warmup_period         = 24*3600,
                    step_period           = 3600)

C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [7]:
env = DiscretizedActionWrapper(env,n_bins_act=1)

In [8]:
log_path =os.path.join("Training", "Logs")
PPO_path = os.path.join("Training" , "Saved Models", "PPO_Boptest")

In [9]:
PPO?

Init signature:
PPO(
    policy: Union[str, type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[dict[str, Any]]

In [10]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path, device="cpu", learning_rate=0.001, gamma= 0.99, batch_size= 64)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
model.learn(total_timesteps=200)

Logging to Training\Logs\PPO_17
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24       |
|    ep_rew_mean     | -1.31    |
| time/              |          |
|    fps             | 0        |
|    iterations      | 1        |
|    time_elapsed    | 3352     |
|    total_timesteps | 2048     |
---------------------------------


In [12]:
#model.save(PPO_path)

In [13]:
#del(model)

In [14]:
#model = PPO.load('PPO_Boptest', env=env)

In [15]:
evaluate_policy(model, env, n_eval_episodes=10)

C:\Users\irmak\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\irmak\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:259: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(-12.053104348946363, 0.0)

In [43]:
from boptestGymEnv import DiscretizedObservationWrapper

In [45]:
# Redefine reward function
class BoptestGymEnvCustomReward(BoptestGymEnv):
    '''Define a custom reward for this building

    '''
    def get_reward(self):
        '''Custom reward function. To expedite learning, we use a clipped reward
        function that has a value of 1 when there is no increase in discomfort
        and 0 otherwise. We use the BOPTEST `GET /kpis` API call to compute the
        total cummulative discomfort from the beginning of the episode. Note
        that this is the true value that BOPTEST uses when evaluating
        controllers.

        '''
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']
        # Calculate objective integrand function as the total discomfort
        objective_integrand = kpis['tdis_tot']
        # Give reward if there is not immediate increment in discomfort
        if objective_integrand == self.objective_integrand:
          reward=1
        else:
          reward=0
        # Record current objective integrand for next evaluation
        self.objective_integrand = objective_integrand
        return reward

In [47]:
lower_setp = 21 + 273.15
upper_setp = 24 + 273.15

In [53]:
import gymnasium as gym
from gym.wrappers import FlattenObservation

In [57]:
env = BoptestGymEnvCustomReward(url                   = url,
                                testcase              = 'bestest_hydronic_heat_pump',
                                actions               = ['oveHeaPumY_u'],
                                observations          = {'reaTZon_y':(lower_setp,upper_setp)},
                                random_start_time     = False,
                                start_time            = 31*24*3600,
                                max_episode_length    = 24*3600,
                                warmup_period         = 24*3600,
                                step_period           = 3600)
env = DiscretizedActionWrapper(env, n_bins_act=1)
env = DiscretizedObservationWrapper(env, n_bins_obs=3, outs_are_bins=True)

done = False
obs, _ = env.reset()

from IPython.display import clear_output
while not done:
  # Clear the display output at each step
  clear_output(wait=True)
  # Compute control signal
  action = model.predict(obs)
  # Print the current operative temperature and decided action
  print('-------------------------------------------------------------------')
  print('State  [Bin #]  = {:.0f}'.format(obs))
  print('Action [ - ]    = {:.0f}'.format(action))
  print('-------------------------------------------------------------------')
  # Implement action
  obs,reward,terminated,truncated,info = env.step(action) # send the action to the environment
  done = (terminated or truncated)

ValueError: Error: Unexpected observation shape () for Box environment, please use (1,) or (n_env, 1) for the observation shape.